# Homework 06 - The Final 

Updating the work from Hw5. 

---
# Loading transformers
Just doing the basics. Copied from Homework 05. 

In [1]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
model = AutoModelForCausalLM.from_pretrained('distilgpt2')

In [3]:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [4]:
print(generator("Once, I started to walk")[0]['generated_text'])

/Users/leiachang/opt/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/Users/leiachang/opt/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Once, I started to walk around the city to see if this was what you wanted. I started to walk around the city to see if this was what you wanted. I started to walk around the city to see if this was what you wanted.


---
# Fine Tuning with Peter Pan 
I think I want to fine-tune this model to fit something a little closer to home. I have a deep love for Peter Pan, so I downloaded the text from Project Gutenberg to work with. Peter Pan (as troublesome as some of the story is) holds themes of everlasting childhood, love for wonder, and imagination. 

File sources are: 
```
"sources/peter-pan.txt"
```

At first I created a truncated text to work with (seen in the cell below), but I realized that the majority of the more interesting, descriptive language occurs later in the text. I decided to try to train using the full text instead, which definitely took longer, but still only about 4 minutes. 

In [5]:
# Create a truncated peter-pan text to work with
with open("sources/truncated-peter-pan.txt", "w") as fh:
    fh.write(open("sources/peter-pan.txt").read()[:20000])

In [6]:
import sys
!{sys.executable} -m pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
import datasets

In [8]:
training_data = datasets.load_dataset('text', data_files="sources/peter-pan.txt")

Found cached dataset text (/Users/leiachang/.cache/huggingface/datasets/text/default-33e0c349e4d6c2f1/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
tokenizer.pad_token = tokenizer.eos_token
tokenized_training_data = training_data.map(
    lambda x: tokenizer(x['text']),
    remove_columns=["text"]
)

Map:   0%|          | 0/6261 [00:00<?, ? examples/s]

In [10]:
block_size = 64
# magic from https://github.com/huggingface/notebooks/blob/master/examples/language_modeling.ipynb
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result
lm_training_data = tokenized_training_data.map(
    group_texts,
    batched=True,
    batch_size=200
)

Map:   0%|          | 0/6261 [00:00<?, ? examples/s]

In [11]:
from transformers import Trainer, TrainingArguments

In [12]:
trainer = Trainer(model=model,
                  train_dataset=lm_training_data['train'],
                  args=TrainingArguments(
                      output_dir='distilgpt2-finetune-peter-pan',
                      num_train_epochs=1,
                      do_train=True,
                      do_eval=False
                  ),
                  tokenizer=tokenizer)

In [13]:
trainer.train()

/Users/leiachang/opt/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=129, training_loss=4.0501832000969, metrics={'train_runtime': 184.2312, 'train_samples_per_second': 5.602, 'train_steps_per_second': 0.7, 'total_flos': 16853640413184.0, 'train_loss': 4.0501832000969, 'epoch': 1.0})

In [14]:
trainer.save_model()

In [15]:
generator("It's time to rest.  ", max_length=100)[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"It's time to rest.         It is the last time that anyone is ever gone and never has been there. I hope you will.”“““”“”“Not really,”“”” said St. John. “I wish you could go back, but I say a little more.”“No,” said Mrs.“Yes, no."

---
# Using the generator

For this iteration, I'm still looking at how I might sift a narrative out of a set of generated texts, each getting more and more randomized/unlikely. My idea is to find a narrative of escape, playing into the themes within Peter Pan of escapism, exploration, and drama. After my experiments the other week, I decided that I wanted to take a heavier hand at editing the output text. 

Below, I generate 5 samples each of a particular temperature and top_k. For each set, I either increase temperature, or the size of the top_k. 

In [28]:
prompt = "I am running away. I am running away, and "

In [29]:
generated_t_1 = []

for x in range(5): 
    generated_t_1.append(generator(prompt, temperature = 1.0, max_length = 50)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [40]:
generated_t_2 = []

for x in range(5): 
    generated_t_2.append(generator(prompt, temperature = 2.0, max_length = 50)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [41]:
generated_k_100 = []

for x in range(5): 
    generated_k_100.append(generator(prompt, top_k = 100, temperature = 2.0, max_length = 50)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [25]:
generated_k_500 = []

for x in range(5): 
    generated_k_500.append(generator(prompt, top_k = 500, temperature = 2.0, max_length = 50)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


--- 
# Adding Formatting 

In [33]:
import random 

In [34]:
to_be_inserted = ["\n", "          ", "\n\n"]

In [37]:
def add_spaces(text): 
    words = text.split()
    maxIndex = len(words)
    chosenIndexes = random.sample(range(0, maxIndex), 5)
    for i in chosenIndexes: 
        words.insert(i, random.choice(to_be_inserted))
    return " ".join(words)

In [38]:
for text in generated_t_1: 
    print(add_spaces(text))
    print("\n\n -------------------------------------\n")

I am running away. I 
 am 
 running away, 
 and !!!” I have come from the last night. “To the rescue?” I am still a 
 long time away!I am not the only one 
 who are looking for me


 -------------------------------------



 I am running away. I            am running away, and --------------!”“Mothers,” I said, “I do not want 

 

 you to 

 be happy.”“I am, Mr. Brien.�


 -------------------------------------

           I am running away. I am running away, and 
 iphante!”He’s trying to think of 

 a dead            man, but to say it is a lie.The wind was very            heavy with them, but they seemed to


 -------------------------------------

I am 

 running            away. I am 
 running away, and                       HAPPY, SISTER.”“Laughter” he cried and asked “Why?”“What did you do?” said the captain


 -------------------------------------

I            am 

 running away. I am running away, and ’Oh my, my sweet 

 bird.”“Now,” he            said, “I can save my life, an

In [42]:
for text in generated_t_2: 
    print(add_spaces(text))
    print("\n\n -------------------------------------\n")

I am 
 running away. 
 I am running away, and _________________Peter - My            wife is quite close at arm in arm but alittle bird 

 by arm by foot in foot. Now for Peter’’s mother Mrs. Jane 
 and Mrs


 -------------------------------------

I am running away. I am 
 running away, and the path there in it!“You can find            one or we all here 

 by walking            us here?“He says 
 the stairs that should look more for your house. His home


 -------------------------------------

I am running away. I am running away, 
 and ************************The music, from it is that song which lives me!Tiffler 

 has 
 now 

 won!Wendy goes. He knows ofthe boy. 

 She doesle with me.


 -------------------------------------

I am running away. I am running away, and ickles 

 on fire. So on his 

 

 way into an ambush...not just for his 
 escape- but now more. Now 

 and they look as he goes after a moment when she stops. What


 -------------------------------------

I am 

 r

In [43]:
for text in generated_k_100: 
    print(add_spaces(text))
    print("\n\n -------------------------------------\n")

I am running away. I am running away, and ________For you boys they            tell Wendy.The door is blown by day.Then 
 their mouth must 
 have drifted off:In 

 this they will 
 keep growing till Peter was as redinned withf


 -------------------------------------

I am running away. I am running away, and ickling in amazement, his mouth open saying            so slowly!That is every small animal in his little short 
 space who once in the world hewaswitching his            love? 
 

 But his very


 -------------------------------------

I am running 

 away. I 
            am running away, and ................................ the moment he arrived on that trail; which in one short moment is that time was on his clock if what 

 was 
 the way was found just before. To this time at last there


 -------------------------------------

I am 

 running away. I am running away, and *************That’s something 
 I’charmed 

 most 

 nights while in the wilderness until            she f

In [44]:
for text in generated_k_500: 
    print(add_spaces(text))
    print("\n\n -------------------------------------\n")

I am running away. I am 
 running away, and ! That 

 is good still: many or 

 
 from the balcony, next Monday, it hardly may grow rich; but keep trying to pull 

 apart first for whereletfor to kiss It then


 -------------------------------------

I am running away. I 

 am running away, and ....................... as 

 Dinkhouse wou had said            Wier now --suddenlyB and they shall pass Jane 
 has hurried home            by-- under his powerDylan‡baring him;gone


 -------------------------------------

I 
 am running away. I am 
 running away, and ****************I WANT TER’ with 
 ALLERSON who let Sarah 
 keep, and amforth upon 

 her hat leaves besidegably my side;what remain amoved this parting of story at it length


 -------------------------------------

I am running away. 

 I am running 

 away, 
 and Hook began!Father what dair answered quite alarm by once weft forward for much the long the nade waitagrosethere put            where Hoomin should've            taug

---
# Conclusion 

Outcomes: 

```
>>>>>> Temp: 1.0 

I am running away. I am 
 running away, 
 and !!!  I am still a long time away. 

I am not the only one who is looking for me. 

                   He cried and asked “Why?”“What did you do?”

and “Oh my, my sweet  bird.” but to say it is a lie.

“Now,” I            said, “I can save my life,  
 by going.”
 
 -------------------------------------
>>>>>> Temp: 2.0 

 
 I am 
 running away. 
 I am running away,  
					He knows of the boy. 

“You can find          all here 
	 by walking“

So on his 
 way into an ambush...not just for his 
 escape- but now more.quite close at arm in arm but a little bird 
 		by arm by foot in foot 
						on fire.

 -------------------------------------
>>>>>> Temp: 2.0, Top-K: 100 

 I am running away. I am running away, and 
then their mouth must 
have drifted off: tickling in amazement, his mouth open saying            
								so slowly
 the moment he arrived on that trail; 

in one short moment
had she been frightened now? Oh.
	”Oh dear”, he            
		 could always think

But there were other visitors there.

-------------------------------------
>>>>>> Temp: 2.0, Top-K: 500 

I am running away. I am 
 running away, and ~~ o of, 

first for whereletfor to kiss
now --suddenly and they shall pass by under his power
baring him;gone

upon her leaves besidegably my side; what remain amoved this parting of story at its length

but afterwards ?             all feeling grief slowlyonme

broke easily 
 		quiet 

 ```
 